In [1]:
if 'google.colab' in str(get_ipython()):
    # authorize & import/mount colab/google drive
    from google.colab import output
    from google.colab import drive
    from google.colab import auth
    auth.authenticate_user()
    drive.mount('/content/gd')

    # install libraries
    !pip install --upgrade openai
    !pip install --upgrade numba
    !pip install transformers
    !pip install sentence_transformers
    !pip install unidecode
    !pip install bertopic
    !pip install unidecode
    !pip install pandas==1.1.5

    # clear output
    output.clear()

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
import openai
pd.set_option('max_colwidth', 300)
warnings.filterwarnings("ignore")
import os

In [2]:
os.chdir("/content/gd/MyDrive/w210_Capstone_Project_Fall2021/Repo/memorai/alice/")

In [32]:
with open('gpt3_api.txt') as f:
    key = f.readlines()[0]

## Prepare Dataset to JSONL

In [ ]:
!openai tools fine_tunes.prepare_data -f /content/gd/MyDrive/w210_Capstone_Project_Fall2021/Repo/memorai/alice/raw_data/alex_combined_data_full_v3.csv

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 711 prompt-completion pairs
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Necessary] Your format `CSV` will be converted to `JSONL`
- [Recommended] Add a whitespace character to the beginning of the completion [Y/n]: y


Your data will be written to a new JSONL file. Proceed [Y/n]: y

Wrote modified file to `/content/gd/MyDrive/w210_Capstone_Project_Fall2021/Repo/memorai/alice/raw_data/alex_combined_data_full_v3_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "/content/gd/MyDrive/w210_Capstone_Project_Fall2021/Repo/memorai/alice/raw_data/alex_combined_data_full_v3_prepared.jsonl"

## Fine tune GPT3 

You can't fine tune a davinci. It says on their doc
> Every fine-tuning job starts from a base model, which defaults to curie. The choice of model influences both the performance of the model and the cost of running your fine-tuned model. Your model can be one of: ada, babbage, or curie. If you would like to fine-tune davinci please reach out to our team.

In [4]:
!openai api fine_tunes.create -t /content/gd/MyDrive/w210_Capstone_Project_Fall2021/Repo/memorai/alice/raw_data/alex_combined_data_full_v3_prepared.jsonl -m davinci --n_epochs 5 --batch_size 5 --learning_rate_multiplier 0.1 --prompt_loss_weight 0.1

Found potentially duplicated files with name '/content/gd/MyDrive/w210_Capstone_Project_Fall2021/Repo/memorai/alice/raw_data/alex_combined_data_full_v3_prepared.jsonl', purpose 'fine-tune' and size 395337 bytes
file-9p0ZUGBnuqI8IEMgShzY4jVD
file-tad5tqVjtYrTpswGqJSqmirN
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 
Uploaded file from /content/gd/MyDrive/w210_Capstone_Project_Fall2021/Repo/memorai/alice/raw_data/alex_combined_data_full_v3_prepared.jsonl: file-qE40deFSOx62aqzby82MfsY5
Created fine-tune: ft-Zo0s216JPRotWhBmJqiQrcQI
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2021-12-01 08:50:13] Created fine-tune: ft-Zo0s216JPRotWhBmJqiQrcQI
[2021-12-01 08:50:25] Fine-tune enqueued. Queue number: 0
[2021-12-01 08:50:30] Fine-tune started
[2021-12-01 08:54:19] Completed epoch 1/5
[2021-12-01 08:56:15] Completed epoch 2/5
[2021-12-01 08:58:10] Completed epoch 3/5
[2021

## Experiment with fine-tuning and completion:

### Fine-tuning hyperparameters
*   **n_epochs**: 5 (fixed)
*   **batch_size**: 4 (default)
*   **learning_rate_multiplier**: 0.01-0.4  
    Definition: When using smaller learning rates, we often recommend training for longer (more epochs). This is likely to lead to better performance.  
Additionally, larger learning rates often play well with larger batch sizes, and smaller learning rates with smaller batch sizes.
*   **prompt_loss_weight**: 0, 0.1, 1  
    Definition: how much the model prioritizes learning from prompt tokens vs learning from completion tokens.

### Completion parameters
* **model**: curie-engine-number  
  Definition: The language model that will be used
* **max_token**: 512 (for stop sequence to work)
  Definition: How much text will be included in the completion
* **temperature**: 0 - 1   
  Definition: Controls the randomness of the result
* **top_p**: 0 - 1  
  Definition: An alternative to Temperature for controlling randomness
* **frequency_penalty**: -2.0 - 2.0  
  Definition: Decreases the model's likelihood of repeating the same line verbatim
* **presence_penalty**: -2.0 - 2.0  
  Definition: Increases the model's likelihood of talking about new topics
* **best_of**:  1 (just want one best completion back)  
  Definition: Setting to only return the best of n completions
* **stop**: [" \###"]  
  Definition: A sequence of characters to end a completion







In [5]:
def completion(prompt, MODEL, TEMP, MAX_TOKEN, FRE_PEN, PRE_PEN):
  """
  Get model response based on completion params
  """
  pronouns = {
      " alex ":"",
      " Alex ":"",
      "Alex ":"",
      "alex ": "",
      "What are your ": "What are my ",
      "what are your ": "what are my ",
      " are you":" am I",
      " are you ":" am I ",
      "Are you ":"Am I ",
      "You ":"I ",
      " you ":" I ",
      " your ":" my ",
      "Your ":"My ",
      " me ":" you ",
      " me":" you"}

  if len(prompt.split()) <3:
    return "That's not a fully formatted question, is it?"

  for key in pronouns.keys():
    prompt = prompt.replace(key, pronouns[key])

  answer = openai.Completion.create(
    echo=True,
    stop=[" \###"],
    model=MODEL,
    temperature=TEMP,
    prompt=prompt,
    max_tokens=MAX_TOKEN,
    presence_penalty=PRE_PEN,
    frequency_penalty=FRE_PEN)
  return answer['choices'][0]['text']

def metadata(get_metadata=False):
  """
  Get total count of engines so far
  If get_metadata, log the engine with finetuning hyperparameters used
  """
  df = pd.DataFrame(columns=['engine_name','n_epochs','batch_size','lr_multiplier','prompt_loss_wt'])
  result = openai.FineTune.list()
  get_engine = result["data"]
  count = 0
  # Need to put hyperparams into a dataframe
  if get_metadata:
    for i in range(len(get_engine)):
      engine_name = get_engine[i]["fine_tuned_model"]
      engine_batch_size = get_engine[i]["hyperparams"]["batch_size"]
      engine_lr_multiplier = get_engine[i]["hyperparams"]["learning_rate_multiplier"]
      engine_n_epoch = get_engine[i]["hyperparams"]["n_epochs"]
      engine_prompt_loss_weight = get_engine[i]["hyperparams"]["prompt_loss_weight"]
      
      metedata_series = pd.Series({'engine_name': engine_name,
                 'n_epochs': engine_n_epoch,
                 'batch_size': engine_batch_size,
                 'lr_multiplier': engine_lr_multiplier,
                 'prompt_loss_wt': engine_prompt_loss_weight})
      df = df.append(metedata_series, ignore_index=True)
    return df
  else:
    for i in get_engine:
      count+=1
    return count


### Prompt completion testing

In [7]:
prompt1="Why do you love climbing?"
prompt2="Why did you free solo El Capitan in Yosemite?"
prompt3="What are some of your favorite adventures"
prompt4="How do you deal with fear when free soloing?"
prompt5="What do you care about most?"

In [ ]:
# Curie
answer_prompt1 = completion(prompt1,"curie:ft-brainmonkey-foundation-2021-10-26-08-56-48", 0.1, 512, 1, 1)
answer_prompt1

"Why do you love climbing? I think that the reason I love climbing is because it allows me to explore and to experience things that are otherwise out of reach. It's a pretty simple answer, but I think that it's true for a lot of people."

In [12]:
# Davinci
answer_prompt1 = completion(prompt1,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt1

"Why do I love climbing? I don't know. I just do."

In [ ]:
# Curie
answer_prompt2 = completion(prompt2,"curie:ft-brainmonkey-foundation-2021-10-26-08-56-48", 0.1, 512, 1, 1)
answer_prompt2

"Why did you free solo El Capitan in Yosemite? What was the motivation? What did you hope to achieve? How'd it feel to climb without a rope?\nBut I think the real question is, why do something like that at all? Why put yourself through such an extreme experience? For me, it was about challenging myself. It's about pushing my own limits. It's about being in a place where nobody else wants to go because it's too dangerous. And then eventually, hopefully, being able to do it without risking life and limb."

In [13]:
# Davinci
answer_prompt2 = completion(prompt2,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt2

"Why did I free solo El Capitan in Yosemite? I mean, I don't know. It's just a challenge."

In [ ]:
# Curie
answer_prompt3 = completion(prompt3,"curie:ft-brainmonkey-foundation-2021-10-26-08-56-48", 0.1, 512, 1, 1)
answer_prompt3

"What are some of your favorite adventures that you've had climbing, that you'd like to share with the listeners? I think my favorite adventure was when I was in my early 20s and I decided that I wanted to solo climb the highest points on all seven continents. And so I set out on what I thought would be a relatively short-term goal of three years, but which turned into an actual 30-year project."

In [14]:
# Davinci
answer_prompt3 = completion(prompt3,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt3

"What are some of my favorite adventures? I mean, the first one that comes to mind is soloing El Cap. That's like my favorite thing ever."

In [ ]:
# Curie
answer_prompt4 = completion(prompt4,"curie:ft-brainmonkey-foundation-2021-10-26-08-56-48", 0.1, 512, 1, 1)
answer_prompt4

"How do you deal with fear when free soloing? I'm not really sure. I think that if you're truly prepared, then it doesn't really matter what your fear is. If you've thought through all the potential scenarios and have a good plan for each one, then it's just a matter of executing. But yeah, I mean there are moments when it does feel quite scary, and that's probably why most people don't do it."

In [15]:
# Davinci 
answer_prompt4 = completion(prompt4,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt4

"How do I deal with fear when free soloing? I think it's just a matter of, like, accepting that you're going to be afraid and then doing it anyway. I mean, I don't know if that makes sense. But basically the way I deal with fear is by telling myself that it's normal to be afraid and then doing it anyway."

In [ ]:
# Curie
answer_prompt5 = completion(prompt5,"curie:ft-brainmonkey-foundation-2021-10-26-08-56-48", 0.1, 512, 1, 1)
answer_prompt5

"What do you care about most?\nBut I think that the thing that I care most about is just making sure that my friends are safe. You know, like making sure that everybody who's involved in climbing is safe. That's kind of up there at the top of my list of things. "

In [16]:
# Davinci 
answer_prompt5 = completion(prompt5,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt5

'What do I care about most? I mean, I think that the thing that I care about most is just like making sure that everybody has a fair shot at life.'

In [ ]:
# Curie
prompt6 = "Who am I?"
answer_prompt6 = completion(prompt6,"curie:ft-brainmonkey-foundation-2021-10-26-08-56-48", 0.1, 512, 1, 1)
answer_prompt6

"Who am I? What's my mission in life? What do I believe in? Those are all good questions, and they're questions that I ask myself every day."

In [17]:
# Davinci 
prompt6 = "Who am I?"
answer_prompt6 = completion(prompt6,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt6

"Who am I? I'm Alex Honnold. I've been climbing for about 10 years now, and in the last few years I've started free soloing, which is basically just climbing without a rope or gear. And that's kind of taken things to a whole new level."

In [ ]:
# Curie
answer_prompt1 = completion(prompt1,"curie:ft-brainmonkey-foundation-2021-11-15-01-40-16", 0, 512, 1, 1)
answer_prompt1

"Why do I love climbing? I think it's because I'm a little bit of an adrenaline junkie."

In [18]:
# Davinci 
answer_prompt1 = completion(prompt1,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0, 512, 1, 1)
answer_prompt1

"Why do I love climbing? I mean, I think it's just the ultimate combination of physical and mental challenge. You know, you're using your body in ways that most humans don't even have the capacity to do and then at the same time you're having to make these decisions under pressure with potentially life or death consequences."

In [19]:
# Davinci 
prompt6 = "Who am I?"
answer_prompt6 = completion(prompt6,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt6

"Who am I? I'm Alex Honnold, and I am currently on a big adventure."

In [ ]:
# Curie
answer_prompt2 = completion(prompt2,"curie:ft-brainmonkey-foundation-2021-11-15-01-40-16", 0.1, 512, 1, 1)
answer_prompt2

"Why did you free solo El Capitan in Yosemite?\n'to be the first to do it, and also because I've always loved big walls."

In [20]:
# Davinci 
answer_prompt2 = completion(prompt2,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt2

"Why did I free solo El Capitan in Yosemite? I mean, I don't know. It's just a challenge. You know, it's like the ultimate test of your own ability to control your mind and body in an environment that is completely foreign and alien."

In [ ]:
# Curie
answer_prompt3 = completion(prompt3,"curie:ft-brainmonkey-foundation-2021-11-15-01-40-16", 0.1, 512, 1, 1)
answer_prompt3

"What are some of your favorite adventures? I mean, you know, like the most epic thing that's ever happened to me is climbing Mount Kilimanjaro. You know, it was a pretty big deal for me because I'd never climbed anything higher than 14,000 feet before and then suddenly there I am on top of Africa at the highest point on Earth."

In [21]:
# Davinci 
answer_prompt3 = completion(prompt3,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt3

"What are some of my favorite adventures? I mean, I think the whole Pier soloing project was pretty rad. Um, yeah, so that's a thing where you're climbing up these piers and then you have to get across gaps between them without falling because if you fall off of one pier it kills your momentum and then you can't make it across the gap. So there's this whole process of like planning out how to do each individual move in such a way that if you fell on any single move it wouldn't kill your momentum so that even if you fell off halfway through the climb or something like that, as long as it didn't kill your speed going across the gap, then basically what happens is when he falls off halfway through he just sort of hangs there for a second and then just drops down into space but doesn't die because his speed is still good enough to carry him all the way over."

In [ ]:
# Curie
answer_prompt4 = completion(prompt4,"curie:ft-brainmonkey-foundation-2021-11-15-01-40-16", 0.1, 512, 1, 1)
answer_prompt4

"How do you deal with fear when free soloing? I mean, I think that's a good question."

In [22]:
# Davinci 
answer_prompt4 = completion(prompt4,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt4

"How do I deal with fear when free soloing? I mean, I guess it's just a matter of controlling your fear. You know, you just have to sort of acknowledge that there is some fear and then go ahead and do it anyway."

In [ ]:
# Curie
prompt5="what do you care about most?"
answer_prompt5 = completion(prompt5,"curie:ft-brainmonkey-foundation-2021-11-15-03-41-34", 0.1, 512, 1, 1)
answer_prompt5

"what do I care about most? I care most about the people in my life. I mean, you know, like who's having a good time and being happy."

In [27]:
# Davinci 
prompt5="what do you care about most?"
answer_prompt5 = completion(prompt5,"davinci:ft-brainmonkey-foundation-2021-12-01-09-02-32", 0.1, 512, 1, 1)
answer_prompt5

"what do I care about most? I mean, I think that the thing that I care about most is just like making sure that everybody has a fair shot. You know, and it's funny because you hear people talk about climate change or whatever and they're like oh well we have to save the Earth for our children. And then you're like but what if your child is blind or something?"

## Check out and export fine-tune engines with hyper-params and completion params

### Checkout our fine-tuned engines

In [26]:
openai.FineTune.list()

<OpenAIObject list at 0x7f2479029470> JSON: {
  "data": [
    {
      "created_at": 1632611085,
      "fine_tuned_model": "curie:ft-brainmonkey-foundation-2021-09-25-23-06-39",
      "hyperparams": {
        "batch_size": 4,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.1,
        "use_packing": null,
        "weight_decay": 0.0
      },
      "id": "ft-OPvNIWkZ8q5l4fw8rbnS6nCH",
      "model": "curie",
      "object": "fine-tune",
      "organization_id": "org-ne81Q44vfYwWClsEbjNqfMhD",
      "result_files": [
        {
          "bytes": 2376,
          "created_at": 1632611201,
          "filename": "compiled_results.csv",
          "id": "file-yoyurg3ZcUpqmngIdJp7JGIv",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 63542,
          "created_at

In [ ]:
# To delete engines
#!openai api models.delete -i curie:ft-brainmonkey-foundation-2021-09-26-09-12-12

In [25]:
engine_metadata = metadata(get_metadata=True)
print(f'Total num of fine tune runs {metadata()}')

Total num of fine tune runs 20


### Export the result for evaluation

In [ ]:
export_finetune = engine_metadata.iloc[10:18]

export_finetune['engine_id'] = [i for i in range(len(export_finetune))]
export_finetune_2 = export_finetune
export_finetune_3 = export_finetune
export_finetune_4 = export_finetune
final_finetune = pd.concat(
    [export_finetune, export_finetune_2,export_finetune_3,export_finetune_4], axis=0).sort_values(
        by=['engine_name']).reset_index(drop=True)

final_finetune["temp"] = [0.1,0.1,0.4,0.4]*8
final_finetune["pres_pen"] = [0.0,1.0,-1.0,-1.5]*8
final_finetune["freq_pen"] = [2.0,1.5,1.5,2.0]*8

final_finetune = final_finetune[['engine_id', 'engine_name','n_epochs','batch_size','lr_multiplier','temp','pres_pen','freq_pen']]

final_finetune.head(8)

,engine_id,engine_name,n_epochs,batch_size,lr_multiplier,temp,pres_pen,freq_pen
0,0,curie:ft-brainmonkey-foundation-2021-10-26-08-56-48,5,5,0.1,0.1,0.0,2.0
1,0,curie:ft-brainmonkey-foundation-2021-10-26-08-56-48,5,5,0.1,0.1,1.0,1.5
2,0,curie:ft-brainmonkey-foundation-2021-10-26-08-56-48,5,5,0.1,0.4,-1.0,1.5
3,0,curie:ft-brainmonkey-foundation-2021-10-26-08-56-48,5,5,0.1,0.4,-1.5,2.0
4,1,curie:ft-brainmonkey-foundation-2021-11-15-01-40-16,5,142,0.1,0.1,0.0,2.0
5,1,curie:ft-brainmonkey-foundation-2021-11-15-01-40-16,5,142,0.1,0.1,1.0,1.5
6,1,curie:ft-brainmonkey-foundation-2021-11-15-01-40-16,5,142,0.1,0.4,-1.0,1.5
7,1,curie:ft-brainmonkey-foundation-2021-11-15-01-40-16,5,142,0.1,0.4,-1.5,2.0


In [ ]:
final_finetune.to_csv("fined_tuned_for_eval.csv", index=False)